In [ ]:
import os
import json

from dotenv import load_dotenv

from utils.googlemaps.api import gm_distance_api, chunks

# LOAD ENV VARIABLES

In [ ]:
load_dotenv()

API_KEY_PATH = os.environ['API_KEY_PATH']
DATA_PATH = os.environ['DATA_PATH']

In [ ]:
with open(os.path.join(API_KEY_PATH, 'api_key.json'), 'r') as file:
    API_KEY = json.load(file)

# GET API DATA

In [ ]:
api_data_time = {}
api_data_dist = {}

for cp, coord in checkpoints.items():
    
    destinations = checkpoints.copy()
    destinations.pop(cp)
    destinations = [x for x in destinations.values()]
    
    time = []
    distance = []
    if len(destinations) > 25:
        
        for chunk_destinations in chunks(destinations, 25):
            
            data = gm_distance_api(coord, chunk_destinations)
            time.extend(data['time'])
            distance.extend(data['distance'])
            
    else:
        
        data = gm_distance_api(coord, destinations)
        time.extend(data['time'])
        distance.extend(data['distance'])
    
    api_data_time[data['origin_addresses'][0]] = time
    api_data_dist[data['origin_addresses'][0]] = distance

# SAVE DATA

In [ ]:
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'api_data_dist_02.json'), 'w') as outfile:
    json.dump(api_data_dist, outfile)
    
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'api_data_time_02.json'), 'w') as outfile:
    json.dump(api_data_time, outfile)
    
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'checkpoints_02.json'), 'w') as outfile:
    json.dump(checkpoints, outfile)